# Step 1: Import necessary libraries


In [2]:
import firebase_admin
from firebase_admin import credentials, db, firestore
import requests
import os

## Import the helpers.py file to our working directory

In [3]:
# url = 'https://raw.githubusercontent.com/StanfordSpezi/SpeziDataPipeline/main/helpers.py'

# # Send a GET request to the URL
# r = requests.get(url)

# # Open a file and write the contents of the response
# with open('helpers.py', 'wb') as f:
#     f.write(r.content)

from helpers import *

# Step 2: Establish Connection to Firebase


In [6]:
# Set your service account credentials
serviceAccountKey_file = 'path/to/serviceAccountKey.json' # This file contains your Firebase project's credentials

# Define your Firebase project ID
project_id = 'spezidatapipelinetemplate' 

collection_name = 'users'

# Check if running in CI environment
if os.getenv('CI') or "FIRESTORE_EMULATOR_HOST" in os.environ:
    
    # Point to the emulator for CI tests
    # os.environ["FIRESTORE_EMULATOR_HOST"] = "localhost:8080"
    os.environ["GCLOUD_PROJECT"] = project_id
    firebase_admin.initialize_app(options={'projectId': project_id})
    db = firestore.Client(project=project_id)
    
    
else: # Connect to the production environment
    if not firebase_admin._apps:  # Check if a Firebase app has already been initialized to prevent reinitialization
        # These credentials allow your application to authenticate with Firebase
        cred = credentials.Certificate(serviceAccountKey_file)
        
        # Initialize your Firebase app with the credentials
        # This step is necessary to interact with Firebase services, including Firestore
        firebase_admin.initialize_app(cred)
        db = firestore.client()
      

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

# Step 3: Access Data on Firestore

## Print Users Summary

In [ ]:
users = export_users_to_csv(db, collection_name, 'users_summary.csv')
users.head()

## Get data from a single HealthKit document of a user

In [ ]:
# Reference to the specified document
doc_ref = db.collection('users')\
    .document('Du3CdrAooIL4sDQajeOt9j0HlPyK')\
    .collection('HealthKit')\
    .document('0960a652-602e-47aa-8381-955d95581922')

doc = doc_ref.get()

if doc.exists:
    doc_dict = doc.to_dict()
    print(doc_dict)
else:
    print("Document not found.")

## Explore Data Quantity Types in the Database

In [ ]:
unique_loinc_codes, display_to_loinc_dict = get_unique_codes_and_displays(db, collection_name)
print(display_to_loinc_dict)

## Fetch and flatten for a selected data type for all users

In [ ]:
selected_code = '55423-8'

flattened_df = fetch_and_flatten_data(db, collection_name, selected_code)   

In [ ]:
flattened_df.head()

# Step 4: Analyze data

## Filter data by removing values outside a predefined range/threshold

In [ ]:
filtered_df = remove_outliers(flattened_df)       

## Aggregate data by date for each user

In [ ]:
daily_df = calculate_daily_data(filtered_df, save_as_csv=True) 

In [ ]:
daily_df.head()

# Plot data for a selected user and export figure

In [ ]:
selected_user = '5okimXt6noCIcD4DGoB13wzXVJjY'
plot_and_export_data(daily_df, date1=None, date2=None, user_id=selected_user, y_lower=None, y_upper=None, same_plot=True, save_as_tif=True)

# Single-line code option: analyze_data() func

In [ ]:
flattened_data, filtered_data, daily_data = analyze_data(db, collection_name)

In [ ]:
daily_data['daily_df_HKQuantityTypeIdentifierDietaryProtein'].head()

# Visualize Data

In [ ]:
vizualize_data(daily_data)